In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("project.db")


In [ ]:
#SQL JOIN

query = """
SELECT
    c.gender,
    c.senior_citizen,
    c.partner,
    c.dependents,

    b.tenure,
    b.monthly_charges,
    b.total_charges,

    s.phone_service,
    s.multiple_lines,
    s.internet_service,
    s.online_security,
    s.online_backup,
    s.device_protection,
    s.tech_support,
    s.streaming_tv,
    s.streaming_movies,

    ctr.contract,
    ctr.paperless_billing,
    ctr.payment_method,

    ch.churn
FROM Customer c
JOIN Billing b ON c.customer_id = b.customer_id
JOIN Services s ON c.customer_id = s.customer_id
JOIN Contract ctr ON c.customer_id = ctr.customer_id
JOIN Churn ch ON c.customer_id = ch.customer_id
"""


I joined normalized tables using customer_id to reconstruct the modeling dataset.

In [ ]:
# Load data

df = pd.read_sql_query(query, conn)
df.head()


,gender,senior_citizen,Partner,Dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,churn
0,Female,0,Yes,No,1,29.85,29.85,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,0
1,Male,0,No,No,34,56.95,1889.50,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,0
2,Male,0,No,No,2,53.85,108.15,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,1
3,Male,0,No,No,45,42.30,1840.75,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),0
4,Female,0,No,No,2,70.70,151.65,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,1


In [28]:
df.shape


(7043, 20)

In [8]:
df["churn"].value_counts(normalize=True)


churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

^ The classes are imbalanced, so we used a stratified train/test split and F1-score.

In [9]:
df.isna().sum()


gender                0
senior_citizen        0
Partner               0
Dependents            0
tenure                0
monthly_charges       0
total_charges        11
phone_service         0
multiple_lines        0
internet_service      0
online_security       0
online_backup         0
device_protection     0
tech_support          0
streaming_tv          0
streaming_movies      0
contract              0
paperless_billing     0
payment_method        0
churn                 0
dtype: int64

In [ ]:
# Handle missing values

df["total_charges"] = df["total_charges"].fillna(df["total_charges"].median())


In [29]:
#Train/test split

from sklearn.model_selection import train_test_split

X = df.drop("churn", axis=1)
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)



Because the churn classes are imbalanced, I used a stratified split to preserve class proportions

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

num_features = ["tenure", "monthly_charges", "total_charges"]
cat_features = X.columns.difference(num_features)


preprocessing = ColumnTransformer([
    ("num", StandardScaler(), num_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
])



Numerical features are scaled, categorical features are one-hot encoded

In [35]:
#Baseline Model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

baseline_model = Pipeline([
    ("prep", preprocessing),
    ("model", LogisticRegression(max_iter=1000))
])

baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print(f"Baseline Logistic Regression F1: {f1_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


Baseline Logistic Regression F1: 0.6040
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1035
           1       0.66      0.56      0.60       374

    accuracy                           0.81      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409



In [34]:
df.describe(include="all").T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
gender,7043,2,Male,3555,NaN,NaN,NaN,NaN,NaN,NaN,NaN
senior_citizen,7043.0,NaN,NaN,NaN,0.162147,0.368612,0.0,0.0,0.0,0.0,1.0
Partner,7043,2,No,3641,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dependents,7043,2,No,4933,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tenure,7043.0,NaN,NaN,NaN,32.371149,24.559481,0.0,9.0,29.0,55.0,72.0
monthly_charges,7043.0,NaN,NaN,NaN,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75
total_charges,7043.0,NaN,NaN,NaN,2281.916928,2265.270398,18.8,402.225,1397.475,3786.6,8684.8
phone_service,7043,2,Yes,6361,NaN,NaN,NaN,NaN,NaN,NaN,NaN
multiple_lines,7043,3,No,3390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
internet_service,7043,3,Fiber optic,3096,NaN,NaN,NaN,NaN,NaN,NaN,NaN


MonthlyCharges shows high variance

Many categorical features dominate

In [ ]:
# Target distribution (visual)
df["churn"].value_counts()
df["churn"].value_counts(normalize=True)

df.groupby("churn")[["tenure", "monthly_charges", "total_charges"]].mean()
pd.crosstab(df["contract"], df["churn"], normalize="index")




churn,0,1
contract,,
Month-to-month,0.572903,0.427097
One year,0.887305,0.112695
Two year,0.971681,0.028319


In [36]:


# Basic dataset overview
df.info()
# Summary statistics
#df.describe(include="all").T
# Missing values
df.isna().sum()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             7043 non-null   object 
 1   senior_citizen     7043 non-null   int64  
 2   Partner            7043 non-null   object 
 3   Dependents         7043 non-null   object 
 4   tenure             7043 non-null   int64  
 5   monthly_charges    7043 non-null   float64
 6   total_charges      7043 non-null   float64
 7   phone_service      7043 non-null   object 
 8   multiple_lines     7043 non-null   object 
 9   internet_service   7043 non-null   object 
 10  online_security    7043 non-null   object 
 11  online_backup      7043 non-null   object 
 12  device_protection  7043 non-null   object 
 13  tech_support       7043 non-null   object 
 14  streaming_tv       7043 non-null   object 
 15  streaming_movies   7043 non-null   object 
 16  contract           7043 

gender               0
senior_citizen       0
Partner              0
Dependents           0
tenure               0
monthly_charges      0
total_charges        0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
contract             0
paperless_billing    0
payment_method       0
churn                0
dtype: int64

Observations from Data Profiling

- The dataset contains a mix of numerical and categorical features, with categorical variables dominating.

- total_charges contains missing values due to customers with short tenure, which were imputed using the median.

- Churn is imbalanced, with fewer churn cases than non-churn cases, motivating the use of F1-score and stratified sampling.

- Tenure shows a strong negative relationship with churn, indicating long-term customers are less likely to churn.

- Monthly charges show a positive relationship with churn, suggesting higher bills increase churn risk.

Data Cleanup Tasks

- Handle missing values in total_charges

- Encode categorical variables using one-hot encoding

- Scale numerical features

- Address class imbalance via evaluation metrics rather than resampling